So i am using Random forest algorithm with 100 trees estimater.
I dropped the following columns while doing model testing 'PassengerId', 'Name', 'Fare','Cabin','Ticket' & I am transforming my object data to numeric values, with filling null values of Age with average and Embarked value to most appearing value.

In [15]:
import cupy as cp
from cuml.ensemble import RandomForestClassifier 
from cuml.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.preprocessing import LabelEncoder

train_data = pd.read_csv('titanic/train.csv')
x_test = pd.read_csv('titanic/test.csv')
y_test = pd.read_csv('titanic/gender_submission.csv')

x_train = train_data.iloc[:,2:]
y_train = train_data.iloc[:,:2]

# print(x_train.columns)
# print(x_test.columns)

# print(y_train.columns)
# print(y_test.columns)


x_train= x_train.drop(columns=['Name', 'Fare','Cabin','Ticket'])
x_test= x_test.drop(columns=['PassengerId', 'Name', 'Fare','Cabin','Ticket'])

y_train=y_train.drop(columns=['PassengerId'])
y_test=y_test.drop(columns=['PassengerId'])

y_test = y_test['Survived']
y_train = y_train['Survived']

le = LabelEncoder()
for col in x_train.select_dtypes(include=['object']).columns:
    x_train[col] = le.fit_transform(x_train[col])
    x_test[col] = le.transform(x_test[col])

x_train['Age'].fillna(x_train['Age'].median(), inplace=True)
x_test['Age'].fillna(x_test['Age'].median(), inplace=True)

# x_train['Cabin'].fillna('N', inplace=True)
# x_test['Cabin'].fillna('N', inplace=True)

x_train['Embarked'].fillna(x_train['Embarked'].mode()[0], inplace=True)
x_test['Embarked'].fillna(x_test['Embarked'].mode()[0], inplace=True)


# print(x_train.head())
# print(y_train.head())


# Create a decision tree classifier
clf = RandomForestClassifier(n_estimators=100)

# Fit the model on the training data
clf.fit(x_train, y_train)

# Predict on the test set
y_pred = clf.predict(x_test)

# Create a DataFrame with PassengerId and Survived columns
submission = pd.DataFrame({
    'PassengerId': x_test.index + 892,
    'Survived': y_pred.astype(int)
})

# Save the DataFrame to a CSV file
submission.to_csv('kaggle_submission.csv', index=False)

# Convert predictions to CPU for accuracy calculation
y_pred_cpu = cp.asnumpy(y_pred)

# Calculate and print accuracy
accuracy = accuracy_score(cp.asnumpy(y_test), y_pred_cpu)
print(f'Accuracy: {accuracy:.2f}')




/home/rajput/miniconda3/envs/pybox_env/lib/python3.8/site-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


Accuracy: 0.94
